In [1]:
import ee

In [2]:
ee.Authenticate()                      # opens a browser for OAuth

True

In [4]:
ee.Initialize(project='western-rider-346015')     # initializes the library

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [5]:
# Simple test: get size of a public vector dataset
fc = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
print('Countries in dataset:', fc.size().getInfo())

# Simple raster stat: mean elevation over a small region
img = ee.Image('CGIAR/SRTM90_V4')
pt = ee.Geometry.Point([77.5946, 12.9716])  # Bengaluru
mean = img.reduceRegion(ee.Reducer.mean(), pt.buffer(5000), 90)
print('Mean elevation (m) near Bengaluru:', mean.get('elevation').getInfo())

Countries in dataset: 312
Mean elevation (m) near Bengaluru: 908.5657533859464


In [6]:
# Create a map object
import geemap
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [13]:
# India boundary
def mask_edge(image):
  edge = image.lt(-30.0)
  masked_image = image.mask().And(edge.Not())
  return image.updateMask(masked_image)


img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VV')
    .map(mask_edge)
)

desc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

spring = ee.Filter.date('2015-03-01', '2015-04-20')
late_spring = ee.Filter.date('2015-04-21', '2015-06-10')
summer = ee.Filter.date('2015-06-11', '2015-08-31')

desc_change = ee.Image.cat(
    desc.filter(spring).mean(),
    desc.filter(late_spring).mean(),
    desc.filter(summer).mean(),
)

asc_change = ee.Image.cat(
    asc.filter(spring).mean(),
    asc.filter(late_spring).mean(),
    asc.filter(summer).mean(),
)

m = geemap.Map()
m.set_center(5.2013, 47.3277, 12)
m.add_layer(asc_change, {'min': -25, 'max': 5}, 'Multi-T Mean ASC', True)
m.add_layer(desc_change, {'min': -25, 'max': 5}, 'Multi-T Mean DESC', True)
m

Map(center=[47.3277, 5.2013], controls=(WidgetControl(options=['position', 'transparent_bg'], position='toprig…

In [15]:
import ee, geemap

ee.Initialize()

gaul = ee.FeatureCollection("FAO/GAUL/2015/level1")

# Select J&K, Himachal, Uttarakhand
states = gaul.filter(
    ee.Filter.Or(
        ee.Filter.eq("ADM1_NAME", "Jammu and Kashmir"),
        ee.Filter.eq("ADM1_NAME", "Himachal Pradesh"),
        ee.Filter.eq("ADM1_NAME", "Uttarakhand")
    )
)
roi = states.geometry()

# ---------------------------
# 2. Sentinel-1 SAR Collection
# ---------------------------
def mask_edge(image):
    edge = image.lt(-30.0)
    masked_image = image.mask().And(edge.Not())
    return image.updateMask(masked_image)

s1 = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filterBounds(roi)
    .filter(ee.Filter.eq("instrumentMode", "IW"))
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV"))
    .select("VV")
    .map(mask_edge)
)

# ---------------------------
# 3. Pre- and Post-flood Images
# ---------------------------
# Example flood event: Aug 2017 (change dates as per case study)
pre_flood = s1.filterDate("2017-07-01", "2017-07-20").mean().clip(roi)
post_flood = s1.filterDate("2017-08-01", "2017-08-20").mean().clip(roi)

# ---------------------------
# 4. Flood Detection
# ---------------------------
# Difference (pre - post) -> water pixels have lower backscatter
diff = pre_flood.subtract(post_flood)

# Thresholding: water = backscatter < -17 dB
flood = post_flood.lt(-17).selfMask()

# ---------------------------
# 5. Visualization
# ---------------------------
m = geemap.Map()
m.center_object(roi, 7)

# Layers
m.add_layer(pre_flood, {"min": -25, "max": 0}, "Pre-Flood VV")
m.add_layer(post_flood, {"min": -25, "max": 0}, "Post-Flood VV")
m.add_layer(diff, {"min": -5, "max": 5, "palette": ["red", "white", "blue"]}, "Backscatter Change")
m.add_layer(flood, {"palette": "blue"}, "Flooded Areas")

m

Map(center=[31.041091627986418, 78.20529314978978], controls=(WidgetControl(options=['position', 'transparent_…

In [17]:
gaul = ee.FeatureCollection("FAO/GAUL/2015/level1")

states = gaul.filter(
    ee.Filter.Or(
        ee.Filter.eq("ADM1_NAME", "Jammu and Kashmir"),
        ee.Filter.eq("ADM1_NAME", "Himachal Pradesh"),
        ee.Filter.eq("ADM1_NAME", "Uttarakhand")
    )
)
roi = states.geometry()

# ---------------------------
# 2. Sentinel-1 VV Collection
# ---------------------------
def mask_edge(image):
    edge = image.lt(-30.0)
    masked_image = image.mask().And(edge.Not())
    return image.updateMask(masked_image)

s1 = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filterBounds(roi)
    .filter(ee.Filter.eq("instrumentMode", "IW"))
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV"))
    .select("VV")
    .map(mask_edge)
)

# ---------------------------
# 3. Pre- and Post-Flood Images
# ---------------------------
# Example flood period: adjust dates
pre_flood = s1.filterDate("2017-07-01", "2017-07-20").mean().clip(roi)
post_flood = s1.filterDate("2017-08-01", "2017-08-20").mean().clip(roi)

# ---------------------------
# 4. Flood Detection
# ---------------------------
# Difference image
diff = pre_flood.subtract(post_flood)

# Flood threshold (SAR water signal < -17 dB)
flood_raw = post_flood.lt(-17).selfMask()

# ---------------------------
# 5. Mask Permanent Water with JRC
# ---------------------------
jrc = ee.Image("JRC/GSW1_4/GlobalSurfaceWater")
permanent = jrc.select("occurrence").gt(80).clip(roi)  # >80% water occurrence = permanent
flood_new = flood_raw.updateMask(permanent.Not())      # remove permanent water

# ---------------------------
# 6. Visualization
# ---------------------------
m = geemap.Map()
m.center_object(roi, 7)

m.add_layer(pre_flood, {"min": -25, "max": 0}, "Pre-Flood VV")
m.add_layer(post_flood, {"min": -25, "max": 0}, "Post-Flood VV")
m.add_layer(diff, {"min": -5, "max": 5, "palette": ["red", "white", "blue"]}, "Backscatter Change")
m.add_layer(flood_raw, {"palette": "blue"}, "Detected Flood (Raw)")
m.add_layer(permanent, {"palette": "cyan"}, "Permanent Water (JRC)")
m.add_layer(flood_new, {"palette": "navy"}, "Flooded Areas (Excluding Permanent Water)")

m

Map(center=[31.041091627986418, 78.20529314978978], controls=(WidgetControl(options=['position', 'transparent_…

In [18]:
gaul = ee.FeatureCollection("FAO/GAUL/2015/level1")

# Target + Neighboring States
target_states = [
    "Jammu and Kashmir",
    "Himachal Pradesh",
    "Uttarakhand",
    "Punjab",
    "Haryana",
    "Uttar Pradesh",
    "Delhi",
    "Ladakh",
    "Chandigarh"
]

states = gaul.filter(ee.Filter.inList("ADM1_NAME", target_states))
roi = states.geometry()

# ---------------------------
# 2. Sentinel-1 VV Collection
# ---------------------------
def mask_edge(image):
    edge = image.lt(-30.0)
    masked_image = image.mask().And(edge.Not())
    return image.updateMask(masked_image)

s1 = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filterBounds(roi)
    .filter(ee.Filter.eq("instrumentMode", "IW"))
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV"))
    .select("VV")
    .map(mask_edge)
)

# ---------------------------
# 3. Pre- and Post-Flood Images
# ---------------------------
pre_flood = s1.filterDate("2017-07-01", "2017-07-20").mean().clip(roi)
post_flood = s1.filterDate("2017-08-01", "2017-08-20").mean().clip(roi)

# ---------------------------
# 4. Flood Detection
# ---------------------------
flood_raw = post_flood.lt(-17).selfMask()

# Mask permanent water using JRC
jrc = ee.Image("JRC/GSW1_4/GlobalSurfaceWater")
permanent = jrc.select("occurrence").gt(80).clip(roi)
flood_new = flood_raw.updateMask(permanent.Not())

# ---------------------------
# 5. Flood Area Calculation (sq.km per state)
# ---------------------------
def calc_area(feature):
    # Pixel area in m²
    flood_area_image = flood_new.multiply(ee.Image.pixelArea())
    # Sum over the feature
    flood_stats = flood_area_image.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=feature.geometry(),
        scale=30,
        maxPixels=1e13
    )
    # Convert to sq.km
    area_sqkm = ee.Number(flood_stats.get("VV")).divide(1e6)
    return feature.set({"Flood_Area_km2": area_sqkm})

flood_stats_fc = states.map(calc_area)

# Export results as a table (optional)
task = ee.batch.Export.table.toDrive(
    collection=flood_stats_fc,
    description="Flood_Area_Per_State",
    folder="GEE_Flood",
    fileFormat="CSV"
)
task.start()

# ---------------------------
# 6. Visualization
# ---------------------------
m = geemap.Map()
m.center_object(roi, 7)

m.add_layer(pre_flood, {"min": -25, "max": 0}, "Pre-Flood VV")
m.add_layer(post_flood, {"min": -25, "max": 0}, "Post-Flood VV")
m.add_layer(flood_new, {"palette": "navy"}, "Flooded Areas (Excluding Permanent Water)")
m.add_layer(states, {"color": "red"}, "State Boundaries")

m

Map(center=[29.33459022294749, 76.87131169638646], controls=(WidgetControl(options=['position', 'transparent_b…

In [20]:
import ee
import datetime
import math
gaul = ee.FeatureCollection("FAO/GAUL/2015/level1")
target_states = [
    "Jammu and Kashmir","Himachal Pradesh","Uttarakhand",
    "Punjab","Haryana","Uttar Pradesh","Delhi","Ladakh","Chandigarh"
]
states_fc = gaul.filter(ee.Filter.inList("ADM1_NAME", target_states))

# --- 2. Helper functions for features ---
def sum_rainfall(start, end, geom):
    # CHIRPS daily precipitation
    col = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filterDate(start, end).select('precipitation')
    sum_img = col.sum()
    stats = sum_img.reduceRegion(ee.Reducer.mean().combine(ee.Reducer.max(), '', True),
                                 geometry=geom, scale=5000, maxPixels=1e13)
    return stats

def mean_smap(start, end, geom):
    col = ee.ImageCollection('NASA_USDA/HSL/SMAP10KM_soil_moisture').filterDate(start, end)
    img = col.mean().select('ssm')  # check band name in your GEE account
    stats = img.reduceRegion(ee.Reducer.mean(), geometry=geom, scale=10000, maxPixels=1e13)
    return stats

def sentinel1_mean(start, end, geom):
    s1 = (ee.ImageCollection("COPERNICUS/S1_GRD")
          .filterDate(start, end)
          .filterBounds(geom)
          .filter(ee.Filter.eq("instrumentMode","IW"))
          .filter(ee.Filter.listContains("transmitterReceiverPolarisation","VV"))
          .select('VV'))
    return s1.mean().reduceRegion(ee.Reducer.mean(), geometry=geom, scale=30, maxPixels=1e13)

def ndvi_mean(start, end, geom):
    s2 = (ee.ImageCollection('COPERNICUS/S2_SR')
          .filterDate(start, end)
          .filterBounds(geom)
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
          .map(lambda i: i.normalizedDifference(['B8','B4']).rename('NDVI')))
    return s2.mean().reduceRegion(ee.Reducer.mean(), geometry=geom, scale=30, maxPixels=1e13)

# Static/topography
srtm = ee.Image('USGS/SRTMGL1_003')
elev_stats = lambda geom: srtm.reduceRegion(ee.Reducer.mean().combine(ee.Reducer.stdDev(), '', True),
                                           geometry=geom, scale=90, maxPixels=1e13)

# JRC permanent water baseline
jrc = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select('occurrence')

# --- 3. Flood label function using sentinel-1 differencing (state-window)
def detect_flood_for_window(start_date, end_date, geom):
    # pre = preceding window of same length
    s = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    e = datetime.datetime.strptime(end_date, '%Y-%m-%d')
    delta = e - s
    pre_end = s - datetime.timedelta(days=1)
    pre_start = pre_end - delta

    pre = (ee.ImageCollection("COPERNICUS/S1_GRD")
           .filterDate(pre_start.strftime('%Y-%m-%d'), pre_end.strftime('%Y-%m-%d'))
           .filterBounds(geom)
           .filter(ee.Filter.eq("instrumentMode","IW"))
           .filter(ee.Filter.listContains("transmitterReceiverPolarisation","VV"))
           .select('VV').mean())

    post = (ee.ImageCollection("COPERNICUS/S1_GRD")
            .filterDate(start_date, end_date)
            .filterBounds(geom)
            .filter(ee.Filter.eq("instrumentMode","IW"))
            .filter(ee.Filter.listContains("transmitterReceiverPolarisation","VV"))
            .select('VV').mean())

    # simple water mask: post < -17 dB and not permanent water
    flood_raw = post.lt(-17)
    permanent = jrc.gt(80).clip(geom)
    flood_new = flood_raw.updateMask(permanent.Not())

    # compute flooded area inside geom
    flood_area = flood_new.multiply(ee.Image.pixelArea()).reduceRegion(
        reducer=ee.Reducer.sum(), geometry=geom, scale=30, maxPixels=1e13)

    # get sum value (in m^2), return 0 if null
    area_m2 = ee.Number(flood_area.get('VV')).max(0)
    return area_m2  # m^2

# --- 4. Build the feature table by iterating windows (monthly) ---
start_year = 2015
end_year = 2020   # choose your period
records = []

for year in range(start_year, end_year+1):
    for month in range(1,13):
        # window: month start to month end
        start = datetime.date(year, month, 1)
        # compute month end
        if month == 12:
            end = datetime.date(year+1, 1, 1) - datetime.timedelta(days=1)
        else:
            end = datetime.date(year, month+1, 1) - datetime.timedelta(days=1)
        start_str = start.strftime('%Y-%m-%d')
        end_str = end.strftime('%Y-%m-%d')

        # For each state, compute features & label
        def map_state(feature):
            geom = feature.geometry()
            # Rainfall sum in window
            rain_stats = sum_rainfall(start_str, end_str, geom)
            rain_mean = ee.Number(rain_stats.get('mean')).float()
            rain_max = ee.Number(rain_stats.get('max')).float()

            # antecedent rainfall (last 30 days)
            ant_start = (start - datetime.timedelta(days=30)).strftime('%Y-%m-%d')
            ant_end = (start - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
            ant_stats = sum_rainfall(ant_start, ant_end, geom)
            ant_mean = ee.Number(ant_stats.get('mean')).float()

            # SMAP mean
            # (NOTE: check correct asset/band; adjust if not available)
            try:
                ssm = mean_smap(start_str, end_str, geom).get('ssm')
            except Exception as e:
                ssm = ee.Number(-9999)

            # Sentinel-1 mean (post window)
            s1_mean = sentinel1_mean(start_str, end_str, geom).get('VV')
            ndvi_m = ndvi_mean(start_str, end_str, geom).get('NDVI')

            # flood area (label)
            area_m2 = detect_flood_for_window(start_str, end_str, geom)

            # static elevation
            elev = elev_stats(geom).get('mean')

            feature = feature.set({
                'year': year,
                'month': month,
                'rain_mean': rain_mean,
                'rain_max': rain_max,
                'ant30_mean': ant_mean,
                'ssm': ssm,
                's1_mean': s1_mean,
                'ndvi_mean': ndvi_m,
                'elev_mean': elev,
                'flood_area_m2': area_m2
            })
            # define binary label if area > 1e6 m2 (1 km2)
            return feature.set('FloodOccur', ee.Number(area_m2).gt(1e6))

        # map across states, add date metadata
        month_fc = states_fc.map(map_state)
        # add property for window
        month_fc = month_fc.map(lambda f: f.set({'window_start': start_str, 'window_end': end_str}))
        # append to an output collection by exporting per-month table
        # Export monthly table to Drive (name includes year-month)
        task = ee.batch.Export.table.toDrive(
            collection=month_fc,
            description=f'features_{year}_{month:02d}',
            folder='GEE_Flood_Features',
            fileNamePrefix=f'features_{year}_{month:02d}',
            fileFormat='CSV'
        )
        task.start()
        print('Started export for', year, month)

Started export for 2015 1
Started export for 2015 2
Started export for 2015 3
Started export for 2015 4
Started export for 2015 5
Started export for 2015 6
Started export for 2015 7
Started export for 2015 8
Started export for 2015 9
Started export for 2015 10
Started export for 2015 11
Started export for 2015 12
Started export for 2016 1
Started export for 2016 2
Started export for 2016 3
Started export for 2016 4
Started export for 2016 5
Started export for 2016 6
Started export for 2016 7
Started export for 2016 8
Started export for 2016 9
Started export for 2016 10
Started export for 2016 11
Started export for 2016 12
Started export for 2017 1
Started export for 2017 2
Started export for 2017 3
Started export for 2017 4
Started export for 2017 5
Started export for 2017 6
Started export for 2017 7
Started export for 2017 8
Started export for 2017 9
Started export for 2017 10
Started export for 2017 11
Started export for 2017 12
Started export for 2018 1
Started export for 2018 2
Sta

In [23]:
import requests

url = "https://mausam.imd.gov.in/api/districtwise_rainfall_api.php?id=164"  # Example: Himachal Pradesh
r = requests.get(url)

print("Status Code:", r.status_code)
print("Raw Response:", r.text[:500])  # print first 500 chars

Status Code: 401
Raw Response: Your IP 122.173.28.97 needs to be whitelisted
